In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.1657,  0.0425,  0.0057, -0.1434, -0.0203, -0.0999,  0.0918,  0.0204,
         -0.1616,  0.0852],
        [-0.0951,  0.0684, -0.0194, -0.1069,  0.0356, -0.0945, -0.0158,  0.0665,
         -0.0463, -0.0347]], grad_fn=<AddmmBackward0>)

5.1.1 自定义块
 在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能：

在下面的代码片段中，我们从零开始编写一个块。 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [6]:
class MLP(nn.Module):
    # 用模型参数声明层，声明两个全联接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
        
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[ 0.1080, -0.0813, -0.1158,  0.1388,  0.0131, -0.0979,  0.0437,  0.0699,
          0.0783, -0.0979],
        [-0.0119, -0.0660,  0.0223,  0.0814,  0.0490, -0.0647, -0.0561,  0.0967,
          0.1759, -0.0005]], grad_fn=<AddmmBackward0>)

5.1.2 顺序块

现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

一种将块逐个追加到列表中的函数。

一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [8]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self,X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.0887,  0.1117,  0.1897,  0.0333,  0.1619,  0.1701,  0.0462, -0.2504,
          0.0388, -0.0942],
        [-0.0559,  0.1014,  0.1072, -0.1140,  0.0719,  0.1864,  0.0597, -0.2485,
         -0.0021, -0.1067]], grad_fn=<AddmmBackward0>)

5.1.3 在前向传播函数中执行代码

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20,20),requires_grad=True)
        self.linear = nn.Linear(20,20)
        
    def forward(self,X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [17]:
net = FixedHiddenMLP()
net(X)

tensor(0.2387, grad_fn=<SumBackward0>)

我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [20]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1401, grad_fn=<SumBackward0>)